# Complex Problem-Solving with Claude 3.7

This notebook explores how to leverage Claude 3.7's extended thinking capability for solving complex technical problems. We'll work through a challenging graph optimization problem that demonstrates:

1. How to break down complex problems into manageable components
2. Using extended thinking effectively for each solution step
3. Validating and visualizing intermediate results
4. Understanding Claude's reasoning process

## Prerequisites

- Basic understanding of Claude 3.7's extended thinking capability (covered in previous lessons)
- Familiarity with Python and basic algorithms
- Understanding of graph theory fundamentals

Let's begin by setting up our environment with the necessary imports.

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx  # For graph visualization
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## The Minimum Spanning Tree with Constraints Problem

We'll tackle a variation of the Minimum Spanning Tree (MST) problem that adds real-world constraints. This problem demonstrates how Claude can help solve complex optimization problems through methodical reasoning.

### Problem Statement:
Given a network of cities that need to be connected with high-speed internet infrastructure:

1. Find the minimum-cost way to connect all cities
2. Subject to these constraints:
   - Maximum distance between any two directly connected cities (fiber optic limitation)
   - Maximum number of connections per city (hub capacity limitation)
   - Certain cities must be connected directly (redundancy requirement)
   - Total budget limitation

This problem combines graph theory, constraint satisfaction, and cost optimization - making it an excellent example for demonstrating Claude's problem-solving capabilities.

Let's first create a visualization of our example network to better understand the problem.

In [ ]:
def create_example_network():
    """
    Create and visualize our example network of cities
    """
    # Create example data
    cities = {
        'A': {'pos': (0, 0), 'population': 500000},
        'B': {'pos': (2, 3), 'population': 300000},
        'C': {'pos': (4, 1), 'population': 400000},
        'D': {'pos': (1, 4), 'population': 200000},
        'E': {'pos': (3, 4), 'population': 350000},
        'F': {'pos': (5, 3), 'population': 250000}
    }
    
    # Create a graph
    G = nx.Graph()
    
    # Add nodes with positions
    for city, data in cities.items():
        G.add_node(city, pos=data['pos'], population=data['population'])
    
    # Add all possible edges with distances as weights
    for city1 in cities:
        for city2 in cities:
            if city1 < city2:  # Avoid duplicate edges
                pos1 = cities[city1]['pos']
                pos2 = cities[city2]['pos']
                distance = ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5
                G.add_edge(city1, city2, weight=round(distance, 2))
    
    # Create the visualization
    plt.figure(figsize=(10, 8))
    pos = nx.get_node_attributes(G, 'pos')
    
    # Draw the network
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)
    nx.draw_networkx_labels(G, pos)
    
    # Draw edges with weights
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edges(G, pos)
    nx.draw_networkx_edge_labels(G, pos, edge_labels)
    
    plt.title("City Network with Distances")
    plt.axis('off')
    plt.show()
    
    return G, cities

# Create and display our example network
network, city_data = create_example_network()

# Display problem constraints
display(Markdown("""
### Problem Constraints:
1. Maximum distance between connected cities: 3.5 units
2. Maximum connections per city: 3
3. Required direct connections: A-B, C-F
4. Total budget: 15 units of distance
"""))

## Breaking Down the Problem with Claude

Before jumping into implementation, let's use Claude's extended thinking capability to:
1. Analyze the problem structure
2. Identify potential solution approaches
3. Plan our implementation strategy
4. Consider potential challenges

We'll start by asking Claude to analyze the problem and suggest a structured approach.

In [ ]:
# First, let's create a detailed prompt that describes our problem
problem_analysis_prompt = """
Please analyze this network optimization problem:

Given a network of 6 cities (A through F) that need to be connected with fiber optic infrastructure:
- Each city must be connected to the network (directly or indirectly)
- The cost of connection is proportional to the distance between cities
- Distances between cities are given in the network visualization above

Constraints:
1. Maximum distance between directly connected cities: 3.5 units
2. Maximum number of connections per city: 3
3. Cities A-B and C-F must have direct connections
4. Total sum of connection distances must not exceed 15 units

Questions to address:
1. What algorithm families might be applicable to this problem?
2. How should we handle the multiple constraints?
3. What approach would you recommend for finding an optimal solution?
4. What potential challenges should we watch out for?

Think through this carefully and provide a structured analysis.
"""

# Get Claude's analysis with extended thinking
analysis_response = claude_utils.invoke_claude(
    bedrock_runtime,
    problem_analysis_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=4096,  # Larger budget for thorough analysis
    max_tokens=2000
)

# Display the response
claude_utils.display_claude_response(analysis_response)

In [ ]:
def validate_constraints(G, city_data, path=None):
    """
    Validate whether a given solution meets all problem constraints
    
    Args:
        G: NetworkX graph representing the solution
        city_data: Dictionary of city information
        path: Optional path to validate
        
    Returns:
        tuple: (bool, list of violations)
    """
    violations = []
    
    # Check if graph is connected
    if not nx.is_connected(G):
        violations.append("Network is not fully connected")
    
    # Check maximum distance constraint
    for (u, v) in G.edges():
        distance = G[u][v]['weight']
        if distance > 3.5:
            violations.append(f"Distance between {u}-{v} ({distance:.2f}) exceeds maximum 3.5")
    
    # Check maximum connections per city
    for node in G.nodes():
        if G.degree(node) > 3:
            violations.append(f"City {node} has {G.degree(node)} connections (max 3)")
    
    # Check required connections
    required = [('A', 'B'), ('C', 'F')]
    for u, v in required:
        if not G.has_edge(u, v):
            violations.append(f"Required connection {u}-{v} is missing")
    
    # Check total budget
    total_distance = sum(G[u][v]['weight'] for (u, v) in G.edges())
    if total_distance > 15:
        violations.append(f"Total distance {total_distance:.2f} exceeds budget of 15")
    
    return len(violations) == 0, violations

In [ ]:
def visualize_solution(G, title="Network Solution"):
    """
    Visualize a proposed solution with highlighted constraints and violations
    
    Args:
        G: NetworkX graph representing the solution
        title: Title for the visualization
    """
    plt.figure(figsize=(12, 8))
    pos = nx.get_node_attributes(G, 'pos')
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)
    nx.draw_networkx_labels(G, pos)
    
    # Draw edges with different colors based on constraints
    edge_colors = []
    for (u, v) in G.edges():
        if (u, v) in [('A', 'B'), ('C', 'F')] or (v, u) in [('A', 'B'), ('C', 'F')]:
            edge_colors.append('green')  # Required connections
        elif G[u][v]['weight'] > 3.5:
            edge_colors.append('red')    # Distance violation
        else:
            edge_colors.append('blue')   # Normal connection
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color=edge_colors)
    
    # Add edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels)
    
    plt.title(title)
    plt.axis('off')
    plt.show()
    
    # Validate and display constraints
    is_valid, violations = validate_constraints(G, city_data)
    if not is_valid:
        print("\nConstraint Violations:")
        for violation in violations:
            print(f"❌ {violation}")
    else:
        print("\n✅ All constraints satisfied")

## Implementing the Solution Strategy

Now that we have Claude's analysis and our validation tools, let's implement a step-by-step solution approach. We'll:
1. Start with a minimum spanning tree as our base solution
2. Modify it to satisfy our constraints
3. Use Claude to guide optimization decisions
4. Validate and visualize each step

Let's begin by generating our initial solution attempt and having Claude analyze it.

In [ ]:
def generate_initial_solution():
    """
    Generate an initial solution using minimum spanning tree
    """
    # Create a new graph with the same nodes
    G_solution = nx.Graph()
    
    # Add nodes with positions
    for city, data in city_data.items():
        G_solution.add_node(city, pos=data['pos'])
    
    # Add required connections first
    required_edges = [('A', 'B'), ('C', 'F')]
    for u, v in required_edges:
        pos1 = city_data[u]['pos']
        pos2 = city_data[v]['pos']
        distance = ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5
        G_solution.add_edge(u, v, weight=round(distance, 2))
    
    # Create MST for remaining connections
    remaining_edges = [(u, v) for u, v in network.edges() if (u, v) not in required_edges]
    remaining_weights = [network[u][v]['weight'] for u, v in remaining_edges]
    
    # Sort edges by weight
    sorted_edges = [x for _, x in sorted(zip(remaining_weights, remaining_edges))]
    
    # Add edges until all cities are connected
    for u, v in sorted_edges:
        if not nx.is_connected(G_solution) or not nx.has_path(G_solution, u, v):
            G_solution.add_edge(u, v, weight=network[u][v]['weight'])
    
    return G_solution

In [ ]:
# After generating and visualizing the initial solution, let's analyze the actual distances

# Generate initial solution
initial_solution = generate_initial_solution()
print("Initial Solution:")
visualize_solution(initial_solution, "Initial Solution (MST-based)")

# Analyze all possible connections and their distances
all_distances = {}
for city1 in city_data:
    for city2 in city_data:
        if city1 < city2:  # Avoid duplicates
            pos1 = city_data[city1]['pos']
            pos2 = city_data[city2]['pos']
            distance = ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5
            all_distances[(city1, city2)] = round(distance, 2)

# Display all distances
print("\nAll possible connections and distances:")
for (city1, city2), distance in sorted(all_distances.items(), key=lambda x: x[1]):
    print(f"{city1}-{city2}: {distance:.2f} units")

# Check A-B distance specifically
a_b_distance = all_distances.get(('A', 'B'), None)
if a_b_distance:
    print(f"\nNote: The A-B connection has distance {a_b_distance:.2f}, which may exceed our constraint.")
    
# Check C-F distance specifically
c_f_distance = all_distances.get(('C', 'F'), None)
if c_f_distance:
    print(f"Note: The C-F connection has distance {c_f_distance:.2f}, which may exceed our constraint.")

### Revised Problem Statement

Based on our analysis, we notice that some of the required direct connections may exceed our ideal maximum distance of 3.5 units.

Let's adjust our constraints to create a more realistic problem:

1. **Primary Constraints:**
   - All cities must be connected (directly or indirectly)
   - Required connections: A-B, C-F must be included (even if they exceed the preferred maximum distance)
   - Maximum connections per city: 3
   - Total budget: 15 units

2. **Secondary Objective:**
   - Minimize the number of connections that exceed 3.5 units
   - Minimize the total distance of all connections

This revised approach acknowledges real-world trade-offs where sometimes strict constraints must be relaxed to find feasible solutions.

In [ ]:
# Let's update our validation function to reflect the revised constraints
def validate_revised_constraints(G, city_data):
    """
    Validate whether a given solution meets our revised problem constraints
    
    Args:
        G: NetworkX graph representing the solution
        city_data: Dictionary of city information
        
    Returns:
        tuple: (bool, list of violations, dict of statistics)
    """
    violations = []
    stats = {
        'long_connections': 0,
        'total_distance': 0,
        'max_connections': 0
    }
    
    # Check if graph is connected
    if not nx.is_connected(G):
        violations.append("Network is not fully connected")
    
    # Check maximum distance constraint and count violations
    for (u, v) in G.edges():
        distance = G[u][v]['weight']
        stats['total_distance'] += distance
        
        if distance > 3.5 and (u, v) not in [('A', 'B'), ('C', 'F')] and (v, u) not in [('A', 'B'), ('C', 'F')]:
            stats['long_connections'] += 1
            violations.append(f"Distance between {u}-{v} ({distance:.2f}) exceeds preferred maximum 3.5")
    
    # Check maximum connections per city
    for node in G.nodes():
        degree = G.degree(node)
        stats['max_connections'] = max(stats['max_connections'], degree)
        if degree > 3:
            violations.append(f"City {node} has {degree} connections (max 3)")
    
    # Check required connections
    required = [('A', 'B'), ('C', 'F')]
    for u, v in required:
        if not G.has_edge(u, v) and not G.has_edge(v, u):
            violations.append(f"Required connection {u}-{v} is missing")
    
    # Check total budget
    if stats['total_distance'] > 15:
        violations.append(f"Total distance {stats['total_distance']:.2f} exceeds budget of 15")
    
    return len(violations) == 0, violations, stats

In [ ]:
# Update our visualization function to reflect the revised approach
def visualize_revised_solution(G, title="Network Solution"):
    """
    Visualize a proposed solution with highlighted constraints and statistics
    
    Args:
        G: NetworkX graph representing the solution
        title: Title for the visualization
    """
    plt.figure(figsize=(12, 8))
    pos = nx.get_node_attributes(G, 'pos')
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)
    nx.draw_networkx_labels(G, pos)
    
    # Draw edges with different colors based on constraints
    edge_colors = []
    for (u, v) in G.edges():
        if ((u, v) in [('A', 'B'), ('C', 'F')] or 
            (v, u) in [('A', 'B'), ('C', 'F')]):
            edge_colors.append('green')  # Required connections
        elif G[u][v]['weight'] > 3.5:
            edge_colors.append('orange')  # Long but allowed connections
        else:
            edge_colors.append('blue')   # Normal connection
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=2)
    
    # Add edge labels
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels)
    
    plt.title(title)
    plt.axis('off')
    plt.show()
    
    # Validate and display constraints
    is_valid, violations, stats = validate_revised_constraints(G, city_data)
    
    print(f"\nSolution Statistics:")
    print(f"- Total Distance: {stats['total_distance']:.2f} units")
    print(f"- Long Connections (>3.5): {stats['long_connections']}")
    print(f"- Maximum Connections per City: {stats['max_connections']}")
    
    if not is_valid:
        print("\nConstraint Violations:")
        for violation in violations:
            print(f"❌ {violation}")
    else:
        print("\n✅ All primary constraints satisfied")

In [ ]:
# Now let's create a revised prompt for Claude that acknowledges our constraint challenges
analysis_prompt = """
Please analyze our network optimization problem with the following considerations:

Given a network of 6 cities (A through F) that need to be connected with fiber optic infrastructure, we need to:
1. Connect all cities (directly or indirectly)
2. Include the required connections A-B and C-F (even though they might exceed our preferred maximum distance)
3. Ensure maximum 3 connections per city
4. Keep the total sum of connection distances under 15 units

We prefer connections to be under 3.5 units of distance, but recognize that some required connections might exceed this.

Please provide a step-by-step approach to building an optimal solution. Specifically:
1. Which algorithm family would be most suitable for this type of constrained optimization?
2. How should we prioritize the constraints?
3. What's a good strategy for minimizing the number of long connections?

Think through this carefully with a focus on practical implementation.
"""

# Get Claude's guidance on our revised approach
approach_response = claude_utils.invoke_claude(
    bedrock_runtime,
    analysis_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=6144,
    max_tokens=2500
)

# Display the response
claude_utils.display_claude_response(approach_response)

In [ ]:
# Let's implement a constraint-based solution algorithm based on Claude's suggestions
def build_constrained_mst():
    """
    Build a constrained minimum spanning tree that prioritizes constraints
    """
    # Create a new graph with the same nodes
    G_solution = nx.Graph()
    
    # Add nodes with positions
    for city, data in city_data.items():
        G_solution.add_node(city, pos=data['pos'])
    
    # Add required connections first (A-B and C-F)
    required_edges = [('A', 'B'), ('C', 'F')]
    for u, v in required_edges:
        pos1 = city_data[u]['pos']
        pos2 = city_data[v]['pos']
        distance = ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5
        G_solution.add_edge(u, v, weight=round(distance, 2))
    
    # Create list of remaining possible edges sorted by weight
    possible_edges = []
    for city1 in city_data:
        for city2 in city_data:
            if city1 < city2:  # Avoid duplicates
                # Skip edges already added
                if G_solution.has_edge(city1, city2):
                    continue
                
                pos1 = city_data[city1]['pos']
                pos2 = city_data[city2]['pos']
                distance = ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5
                possible_edges.append((city1, city2, round(distance, 2)))
    
    # Sort by distance
    possible_edges.sort(key=lambda x: x[2])
    
    # Add edges until all cities are connected, respecting constraints
    for u, v, weight in possible_edges:
        # Skip if this would exceed 3 connections for either city
        if G_solution.degree(u) >= 3 or G_solution.degree(v) >= 3:
            continue
            
        # Skip if already connected unless this creates a useful shortcut without creating a cycle
        if nx.has_path(G_solution, u, v):
            continue
            
        # Add the edge
        G_solution.add_edge(u, v, weight=weight)
        
        # If we've connected all cities, stop
        if nx.is_connected(G_solution) and all(G_solution.has_node(city) for city in city_data):
            break
    
    return G_solution

# Build our solution following this approach
constrained_solution = build_constrained_mst()
print("Constrained MST Solution:")
visualize_revised_solution(constrained_solution, "Constrained MST-based Solution")

In [ ]:
# Based on Claude's analysis, let's try to optimize our solution further
def optimize_solution(G, max_iterations=5):
    """
    Optimize the solution by making incremental improvements
    """
    best_G = G.copy()
    best_stats = validate_revised_constraints(best_G, city_data)[2]
    
    # Try a series of improvements
    for i in range(max_iterations):
        # Try removing an edge and adding a better one
        current_G = best_G.copy()
        
        # Find all non-required edges sorted by distance (longest first)
        removable_edges = [(u, v) for u, v in current_G.edges() 
                          if (u, v) not in [('A', 'B'), ('C', 'F')] 
                          and (v, u) not in [('A', 'B'), ('C', 'F')]]
        removable_edges = sorted(removable_edges, 
                                key=lambda x: current_G[x[0]][x[1]]['weight'], 
                                reverse=True)
        
        improved = False
        
        # Try removing each edge
        for edge_to_remove in removable_edges:
            temp_G = current_G.copy()
            temp_G.remove_edge(*edge_to_remove)
            
            # If removing this edge disconnects the graph, we need to add a replacement
            if not nx.is_connected(temp_G):
                # Find the best edge to reconnect the graph
                u, v = edge_to_remove
                components = list(nx.connected_components(temp_G))
                
                # Find which components our disconnected nodes are in
                u_component = None
                v_component = None
                for comp in components:
                    if u in comp:
                        u_component = comp
                    if v in comp:
                        v_component = comp
                
                # Look for alternative edges between components
                candidate_edges = []
                for node1 in u_component:
                    for node2 in v_component:
                        if node1 != node2 and not temp_G.has_edge(node1, node2):
                            # Check degree constraint
                            if temp_G.degree(node1) < 3 and temp_G.degree(node2) < 3:
                                pos1 = city_data[node1]['pos']
                                pos2 = city_data[node2]['pos']
                                distance = ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5
                                candidate_edges.append((node1, node2, round(distance, 2)))
                
                # Add the shortest alternative edge
                if candidate_edges:
                    candidate_edges.sort(key=lambda x: x[2])
                    best_alt = candidate_edges[0]
                    temp_G.add_edge(best_alt[0], best_alt[1], weight=best_alt[2])
                    
                    # Check if this improves the solution
                    temp_valid, temp_violations, temp_stats = validate_revised_constraints(temp_G, city_data)
                    
                    # If valid and better than our current best, update
                    if temp_valid and temp_stats['total_distance'] < best_stats['total_distance']:
                        best_G = temp_G.copy()
                        best_stats = temp_stats
                        improved = True
                        print(f"Iteration {i+1}: Improved by replacing {edge_to_remove} with {(best_alt[0], best_alt[1])}")
                        break
        
        # If no improvements found, stop
        if not improved:
            print(f"No further improvements found after {i+1} iterations")
            break
    
    return best_G

# Try to optimize our solution
final_solution = optimize_solution(constrained_solution)
print("\nOptimized Solution:")
visualize_revised_solution(final_solution, "Optimized Network Solution")

In [ ]:
# Now let's ask Claude to analyze our final solution and extract key insights
final_edges = list(final_solution.edges(data=True))
final_edge_info = "\n".join([f"- {u}-{v}: distance = {d['weight']:.2f}" 
                           for u, v, d in final_edges])

valid, violations, stats = validate_revised_constraints(final_solution, city_data)

final_analysis_prompt = f"""
Please analyze our final network solution:

Final connections:
{final_edge_info}

Solution statistics:
- Total distance: {stats['total_distance']:.2f} units
- Long connections (>3.5): {stats['long_connections']}
- Maximum connections per city: {stats['max_connections']}

Please provide:
1. A comprehensive analysis of this solution's quality
2. The key tradeoffs made in this solution
3. Practical insights for handling multi-constraint optimization problems
4. How would you approach similar problems in other domains?

Structure your analysis to highlight both the technical and practical aspects of this solution approach.
"""

# Get Claude's analysis of final solution
final_analysis_response = claude_utils.invoke_claude(
    bedrock_runtime,
    final_analysis_prompt,
    CLAUDE_37_SONNET_MODEL_ID,
    enable_reasoning=True,
    reasoning_budget=8192,
    max_tokens=3000
)

# Display the response
claude_utils.display_claude_response(final_analysis_response)

## Key Insights from Complex Problem-Solving with Claude 3.7

Throughout this lesson, we've tackled a complex constrained optimization problem by combining algorithmic approaches with Claude 3.7's extended thinking capabilities. Here are the key takeaways:

### 1. Breaking Down Complex Problems

Complex problems become manageable when broken into smaller components:
- Define clear constraints and requirements
- Recognize which constraints are hard (must be met) vs. soft (preferences)
- Start with a basic algorithm as foundation (Minimum Spanning Tree in our case)
- Incrementally improve while monitoring constraint satisfaction

### 2. Leveraging Extended Thinking for Solution Strategy

Claude's extended thinking capability proved valuable for:
- Analyzing problem structure and identifying solution approaches
- Considering potential algorithm families and their applicability
- Evaluating tradeoffs between different constraints
- Suggesting improvement strategies for our initial solutions

### 3. Handling Real-World Complexity

Real-world problems often require balancing competing constraints:
- Sometimes perfect solutions don't exist - compromises are necessary
- Prioritizing constraints is essential for finding practical solutions
- Incremental improvements can yield significant gains
- Visualization helps understand the implications of different design choices

### 4. The Role of Human-AI Collaboration

The most effective approach combined:
- Algorithm-based initial solutions (human-designed code)
- AI analysis of constraint violations (Claude's extended thinking)
- Human judgment about which constraints to prioritize
- Iterative improvements guided by both human and AI insights

## Conclusion: Applying These Techniques to Your Own Problems

The approaches we've explored in this lesson can be applied to a wide range of complex optimization problems:

- **Supply Chain Optimization**: Determining warehouse locations, shipping routes, and inventory distribution
- **Resource Allocation**: Allocating limited resources across competing projects or departments
- **Network Design**: Creating communication networks, transportation systems, or utility grids
- **Scheduling Problems**: Allocating tasks to resources while satisfying time and capacity constraints

When tackling your own complex problems, remember these key principles:

1. **Start with a clear problem formulation**: Define variables, constraints, and objectives
2. **Use extended thinking to analyze the problem structure**: Before diving into code
3. **Begin with a feasible solution**: Even if sub-optimal, it provides a foundation
4. **Iteratively improve**: Make incremental changes that respect constraints
5. **Visualize intermediate results**: To gain insights and detect issues early
6. **Balance competing constraints**: Recognize when perfect solutions aren't possible

By combining algorithmic approaches with Claude's extended thinking capabilities, you can tackle increasingly complex problems that would be difficult to solve with either approach alone.

In the next lesson, we'll explore techniques for migrating existing workloads to leverage Claude 3.7's capabilities.